# Trampoline Competition Scheduler

## Breakdown of Data Importing and Processing
### 1. Data Importing

Via Pandas, we can import data from any Excel-based file, including TrampOnline's .xls files, convert it into a .csv file, and view it. This section deals with the import process.

In [1]:
# Import required packages.

# Import pandas for data analysis.
import pandas as pd

In [2]:
# Read the TrampOnline .xls Excel file.
df = pd.read_excel("data/TrampOnline_Sample_Competitors.xls", sheet_name = "Sheet1", header = 0)

# Convert the .xls file into a .csv file.
df.to_csv("data/TrampOnline_Sample_Competitors.csv", index = False)

In [3]:
# Read the .csv file with all the entries.
df = pd.read_csv("data/TrampOnline_Sample_Competitors.csv", keep_default_na = True, delimiter = ",", skipinitialspace = True, encoding = "utf-8-sig")

### 2. Data Cleaning

The data should be checked and ensured to not have been corrupted or similar. This section performs some sanity checks on the data and then standardises the types.

In [4]:
# Find the shape of the data in the format (number of rows, number of columns).
df.shape

(82, 11)

In [5]:
# Display the first five results, starting at index zero.
df.head(5)

,ID,Name,Club,ClassName,StartOrder,Discipline,Team,Team_Category,Guest,flight,photo_consent
0,10134.0,Derbiled Áed,DCU,Novice Women,20.0,TRI,NaN,NaN,0.0,1.0,1.0
1,10304.0,Anwar Fateh,MU,Novice Men,12.0,TRI,NaN,NaN,0.0,1.0,1.0
2,10403.0,Karpos Pankratios,QUB,Intermediate Men,1.0,TRI,NaN,NaN,0.0,1.0,1.0
3,10999.0,Sören Shiva,TCD,Intervanced Men,45.0,TRI,A,NaN,0.0,NaN,1.0
4,11568.0,Yolotzin Thibaut,UCC,Intermediate Women,65.0,TRI,NaN,NaN,0.0,NaN,1.0


In [6]:
# Display the last five results.
df.tail(5)

,ID,Name,Club,ClassName,StartOrder,Discipline,Team,Team_Category,Guest,flight,photo_consent
77,NaN,TEst11,UCC,Novice Women,NaN,TRI,NaN,NaN,NaN,NaN,NaN
78,NaN,Test12,UCD,Novice Women,NaN,TRI,NaN,NaN,NaN,NaN,NaN
79,NaN,Test13,MU,Novice Women,NaN,TRI,NaN,NaN,NaN,NaN,NaN
80,NaN,Test14,TCD,Novice Women,NaN,TRI,NaN,NaN,NaN,NaN,NaN
81,NaN,Test15,TCD,Novice Women,NaN,TRI,NaN,NaN,NaN,NaN,NaN


In [7]:
# Check the types of the .csv file.
df.dtypes

ID               float64
Name              object
Club              object
ClassName         object
StartOrder       float64
Discipline        object
Team              object
Team_Category    float64
Guest            float64
flight           float64
photo_consent    float64
dtype: object

Based on the above data types, the following modifications should be made.

- **ID:** Perfectly fine as an integer.

- **Name:** Should be converted into a string.

- **Club:** Should be converted into a category.

- **ClassName:** Should be converted into a category.

- **StartOrder:** Perfectly fine as an integer.

- **Discipline:** Should be converted into a category.

- **Team:** Should be converted into a string.

- **Team_Category:** Should be converted into a category.

- **Guest:** Should be converted into a Boolean.

- **flight:** Perfectly fine as a float.

- **photo_consent:** Should be converted into a Boolean.

In [8]:
# Change all data fields to the appropriate type.
df["Name"] = df["Name"].astype("string")
df["Club"] = df["Club"].astype("category")
df["ClassName"] = df["ClassName"].astype("category")
df["Discipline"] = df["Discipline"].astype("category")
df["Team"] = df["Team"].astype("string")
df["Team_Category"] = df["Team_Category"].astype("category")
df["Guest"] = df["Guest"].astype("boolean")
df["photo_consent"] = df["photo_consent"].astype("boolean")

#### Sanity Checks

This section checks for duplicate rows and the number of unique columns.

In [9]:
# Check for duplicate columns.
# If the result is "Empty DataFrame", then there are no duplicates.
print(df[df.duplicated()])

Empty DataFrame
Columns: [ID, Name, Club, ClassName, StartOrder, Discipline, Team, Team_Category, Guest, flight, photo_consent]
Index: []


In [10]:
# Check for the number of unique values in each column. In big competitions, the number of trampoline, DMT, and tumbling competitor categories
# should equal the number of levels plus one to account for anyone not competing.
df.nunique()

ID               25
Name             41
Club              8
ClassName        22
StartOrder       21
Discipline        3
Team              1
Team_Category     0
Guest             2
flight            2
photo_consent     1
dtype: int64

In [11]:
# Save the updated data frame back to CSV.
df.to_csv("data/Updated_Sample_TrampOnline_Data.csv", index = False)

## Competitor Scheduling

The following section will get into organising the competitors into their groups.

There are seven trampoline categories to consider: Novice, Intermediate, Intervanced, Advanced, Elite, Elite-Pro, and Disability (any category). Competitors should be separated based on these levels, and then again by their category. The only exception to the category separation rule is *"Disability (any category)"* as it is not usually split into Men and Women categories.

Once this is done, a check should be done to investigate the number of competitors in categories and decide on how many flights are appropriate. A general rule of thumb is 12 competitors per flight, however this isn't a hard rule. There may be more or less.

### Individual Trampoline (TRI), Double-Mini Trampoline (DMT), Tumbling (TUM), and Synchronised Trampoline (TRS) Level Ordering

The first step is to organise competitors by level. The levels should be put into a specific order (generally from lowest to highest), and then competitors should be split by category (only in TRI).

In [12]:
# Define the order of individual trampoline (TRI) levels.
trampoline_level_order = ["Novice", "Intermediate", "Intervanced", "Advanced", "Elite", "Elite-Pro", "Disability (any category)"]

# Define the order of double-mini-trampoline (DMT) levels.
dmt_level_order = ["DMT Level 1", "DMT Level 2", "DMT Level 3", "DMT Level 4", "DMT Level 5", "DMT Level 6"]

# Define the order of tumbling (TUM) levels.
tum_level_order = ["Tumbling Level 1", "Tumbling Level 2", "Tumbling Level 3", "Tumbling Level 4"]

# Define the order of synchronised trampoline (TRS) levels.
trs_level_order = ["Lower Synchro", "Higher Synchro", "Synchro Level 1", "Synchro Level 2", "Synchro Level 3"]

# ---------------------- TRI PROCESSING ---------------------- #

# Filter for individual trampoline (TRI) competitors.
tri_df = df[df["Discipline"] == "TRI"].copy()

# Split "ClassName" into level and category.
tri_df[["tri_level", "tri_category"]] = tri_df["ClassName"].str.extract(r"^(.*?)\s+(Men|Women)$")

# Convert level into an ordered categorical.
tri_df["tri_level"] = pd.Categorical(tri_df["tri_level"], categories = trampoline_level_order, ordered = True)

# Sort by level and category.
tri_df = tri_df.sort_values(by = ["tri_level", "tri_category", "Name", "Club"]).reset_index(drop = True)

# Remove the "tri_level" and "tri_category" columns.
tri_df = tri_df.drop(columns = ["tri_level", "tri_category"])

# ---------------------- DMT PROCESSING ---------------------- #

# Filter for double-mini trampoline (DMT) competitors.
dmt_df = df[df["Discipline"] == "DMT"].copy()

# Extract DMT level directly from ClassName.
dmt_df["dmt_level"] = pd.Categorical(dmt_df["ClassName"], categories = dmt_level_order, ordered = True)

# Sort by DMT level, then name and club.
dmt_df = dmt_df.sort_values(by = ["dmt_level", "Name", "Club"]).reset_index(drop = True)

# Remove the "dmt_level" column.
dmt_df = dmt_df.drop(columns = ["dmt_level"])

# ---------------------- TUM PROCESSING ---------------------- #

# Filter for tumbling (TUM) competitors.
tum_df = df[df["Discipline"] == "TUM"].copy()

# Extract TUM level directly from ClassName.
tum_df["tum_level"] = pd.Categorical(tum_df["ClassName"], categories = tum_level_order, ordered = True)

# Sort by TUM level, then name and club.
tum_df = tum_df.sort_values(by = ["tum_level", "Name", "Club"]).reset_index(drop = True)

# Remove the "tum_level" column.
tum_df = tum_df.drop(columns = ["tum_level"])

# ---------------------- TRS PROCESSING ---------------------- #

# Filter for synchronised trampoline (TRS) competitors.
trs_df = df[df["Discipline"] == "TRS"].copy()

# Extract TRS level directly from ClassName.
trs_df["trs_level"] = pd.Categorical(trs_df["ClassName"], categories = trs_level_order, ordered = True)

# Sort by TRS level, then name and club.
trs_df = trs_df.sort_values(by = ["trs_level", "Name", "Club"]).reset_index(drop = True)

# Remove the "trs_level" column.
trs_df = trs_df.drop(columns = ["trs_level"])

# ---------------------- MERGE AND OUTPUT ---------------------- #

# Get all other competitors, in case there are any not accounted for (not TRI, DMT, TUM, or TRS).
other_df = df[~df["Discipline"].isin(["TRI", "DMT", "TUM", "TRS"])]

# Combine all the disciplines in the defined order.
df = pd.concat([tri_df, dmt_df, tum_df, trs_df, other_df], ignore_index = True)

# Print the sorted dataset.
print(df)

         ID               Name    Club         ClassName  StartOrder  \
0   13001.0    Antwerp Flazgod     DCU        Novice Men         3.0   
1   12999.0    Antwerp Flazgod  Exeter        Novice Men         2.0   
2   10304.0        Anwar Fateh      MU        Novice Men        12.0   
3       NaN             TEst11     UCC        Novice Men         NaN   
4       NaN              Test1     UCC        Novice Men         NaN   
..      ...                ...     ...               ...         ...   
77  13020.0  Karpos Pankratios     QUB       DMT Level 6        29.0   
78  13055.0    Antwerp Flazgod  Exeter  Tumbling Level 1         5.0   
79  13045.0    Shoshanna Assol      UL  Tumbling Level 2        99.0   
80  13052.0       Echo Longray     TCD  Tumbling Level 3        11.0   
81  13048.0             Akma 2     TCD  Tumbling Level 4        14.0   

   Discipline  Team Team_Category  Guest  flight  photo_consent  
0         TRI  <NA>           NaN  False     1.0           True  
1  

The below code will transform the data frame into a block-formatted table structured for Excel.

In [13]:
def format_discipline_sheet(df_subset, add_plus=False):
    if df_subset.empty:
        return pd.DataFrame(), [], []

    class_order = df_subset["ClassName"].drop_duplicates()
    group_blocks = []
    level_titles = []
    group_sizes = []

    for class_name in class_order:
        class_display = class_name.replace("Women", "Ladies")
        if add_plus:
            class_display += " +"
        level_titles.append(class_display)

        group = df_subset[df_subset["ClassName"] == class_name].reset_index(drop=True)
        group.insert(0, "#", range(1, len(group) + 1))
        group = group[["#", "Name", "Club"]]

        flights = split_into_flights(group)
        group_sizes.append(sum(len(f) for f in flights))

        # === Formatting rows ===
        level_blank = pd.DataFrame([["", "", ""]], columns=group.columns)
        col_headers = pd.DataFrame([["#", "Name", "Club"]], columns=group.columns)
        spacer = pd.DataFrame([["", "", ""]], columns=group.columns)

        full_block = pd.concat([level_blank, col_headers, spacer], ignore_index=True)

        for i, flight_df in enumerate(flights, start=1):
            empty_row = pd.DataFrame([["", "", ""]], columns=group.columns)
            flight_header = pd.DataFrame([[f"Flight {i}", "", ""]], columns=group.columns)
            full_block = pd.concat([full_block, empty_row, flight_header, flight_df], ignore_index=True)

        group_blocks.append(full_block)

    # Match height across blocks for alignment
    max_height = max(len(g) for g in group_blocks)
    for i, block in enumerate(group_blocks):
        group_blocks[i] = block.reindex(range(max_height)).reset_index(drop=True)

    # Add horizontal spacers between blocks
    spaced_blocks = []
    for i, block in enumerate(group_blocks):
        spaced_blocks.append(block)
        if i < len(group_blocks) - 1:
            spacer_col = pd.DataFrame([""] * max_height, columns=[""])
            spaced_blocks.append(spacer_col)

    final_df = pd.concat(spaced_blocks, axis=1)
    return final_df, level_titles, group_sizes

def split_into_flights(df, max_flight_size=12):
    total = len(df)
    if total <= 15:
        return [df.copy()]

    n_flights = (total - 1) // max_flight_size + 1

    base = total // n_flights
    extras = total % n_flights

    flights = []
    start = 0
    for i in range(n_flights):
        size = base + (1 if i < extras else 0)
        flights.append(df.iloc[start:start+size].copy())
        start += size

    return flights

The below code is responsible for the writing operations to the Excel file itself.

In [14]:
def write_formatted(sheet_name, df_subset, writer, add_plus=False):
    workbook = writer.book
    formatted_df, levels, group_sizes = format_discipline_sheet(df_subset, add_plus)
    if formatted_df.empty:
        return

    startrow = 3
    startcol = 1

    formatted_df.to_excel(
        writer, sheet_name=sheet_name, index=False, header=False,
        startrow=startrow, startcol=startcol
    )
    worksheet = writer.sheets[sheet_name]

    # === Formats ===
    level_header_format = workbook.add_format({
        "bold": True, "align": "center", "valign": "vcenter",
        "bg_color": "#D9E1F2", "border": 2
    })

    column_header_format = workbook.add_format({
        "bold": True, "align": "center", "valign": "vcenter", "border": 2
    })

    flight_header_format = workbook.add_format({
        "bold": True, "align": "center", "valign": "vcenter",
        "bg_color": "#D9E1F2", "border": 2
    })

    bright_blue = workbook.add_format({"bg_color": "#D9E1F2"})
    title_format = workbook.add_format({
        "bold": True, "font_size": 16, "align": "center",
        "valign": "vcenter", "bg_color": "#D9E1F2", "border": 2
    })

    title_map = {
        "TRA Flights": "TRAMPOLINE INDIVIDUAL COMPETITORS",
        "DMT Flights": "DOUBLE-MINI TRAMPOLINE COMPETITORS",
        "TUM Flights": "TUMBLING COMPETITORS",
        "SYN Flights": "SYNCHRONISED TRAMPOLINE COMPETITORS"
    }
    sheet_title = title_map.get(sheet_name, "COMPETITORS")
    total_cols = startcol + formatted_df.shape[1]

    # Fill title background
    for row in range(2):
        for col in range(startcol, total_cols):
            worksheet.write(row, col, "", bright_blue)

    worksheet.merge_range(0, startcol, 1, total_cols - 1, sheet_title, title_format)

    # === Per-level Formatting ===
    col = 0
    level_index = 0
    while col < formatted_df.shape[1]:
        if col + 2 >= formatted_df.shape[1]:
            break

        abs_col_start = startcol + col
        abs_col_end = abs_col_start + 2

        # Write level header (first non-empty row)
        level_row = startrow
        worksheet.merge_range(level_row, abs_col_start, level_row, abs_col_end,
                              levels[level_index], level_header_format)

        # Write column headers (second non-empty row)
        col_header_row = level_row + 1
        for c in range(abs_col_start, abs_col_end + 1):
            val = formatted_df.iloc[1, col + (c - abs_col_start)]
            worksheet.write(col_header_row, c, val, column_header_format)

        # Start from the 3rd data row (after level + column + spacer)
        row_ptr = startrow + 3
        max_row = len(formatted_df)

        while row_ptr < max_row:
            val = formatted_df.iat[row_ptr - startrow, col]
            if isinstance(val, str) and val.startswith("Flight"):
                # Apply flight header formatting
                worksheet.merge_range(row_ptr, abs_col_start, row_ptr, abs_col_end,
                                      val, flight_header_format)

                # Find competitors below flight header
                comp_row = row_ptr + 1
                while (comp_row < max_row and
                       pd.notna(formatted_df.iat[comp_row - startrow, col]) and
                       not str(formatted_df.iat[comp_row - startrow, col]).startswith("Flight")):

                    for c in range(abs_col_start, abs_col_end + 1):
                        rel_r = comp_row - startrow
                        rel_c = c - startcol
                        val = formatted_df.iat[rel_r, rel_c]

                        border_format = workbook.add_format()
                        if comp_row == row_ptr + 1:
                            border_format.set_top(2)
                        # Check if next row is empty or a new flight
                        next_val = (formatted_df.iat[comp_row + 1 - startrow, col]
                                    if comp_row + 1 < max_row else "")
                        if (pd.isna(next_val) or str(next_val).startswith("Flight") or str(next_val).strip() == ""):
                            border_format.set_bottom(2)
                        if c == abs_col_start:
                            border_format.set_left(2)
                        if c == abs_col_end:
                            border_format.set_right(2)

                        worksheet.write(comp_row, c, val if pd.notna(val) else "", border_format)

                    comp_row += 1
                row_ptr = comp_row
            else:
                row_ptr += 1

        level_index += 1
        col += 4  # 3 columns + spacer

    # === Resize Columns ===
    for i in range(formatted_df.shape[1]):
        excel_col = startcol + i
        col_data = formatted_df.iloc[:, i].astype(str)

        if col_data.str.strip().eq("").all():
            worksheet.set_column(excel_col, excel_col, 1)
        elif formatted_df.columns[i] == "#":
            worksheet.set_column(excel_col, excel_col, 3.5)  # Force narrow #
        else:
            max_len = col_data.map(len).max()
            adjusted_width = min(max(5, max_len + 1), 30)
            worksheet.set_column(excel_col, excel_col, adjusted_width)

    worksheet.set_column(0, 0, 1)  # Pad left edge

The below code executes the writing of data into thr Excel file.

In [15]:
with pd.ExcelWriter("data/Grouped_Competitors_by_Discipline.xlsx", engine="xlsxwriter") as writer:
    write_formatted("TRA Flights", tri_df, writer, add_plus=True)
    write_formatted("DMT Flights", dmt_df, writer, add_plus=False)
    write_formatted("TUM Flights", tum_df, writer, add_plus=False)
    write_formatted("SYN Flights", trs_df, writer, add_plus=False)

print("Excel file written successfully.")

Excel file written successfully.
